<!-- File automatically generated using DocOnce (https://github.com/doconce/doconce/):
doconce format ipynb Networks.do.txt  -->

## Networks
As the field of computational neuroscience continues to evolve, the necessity to comprehend 
and simulate the dynamic interactions within neural networks becomes increasingly important. 
Network dynamics, which involves understanding how various interconnected nodes in a network 
interact over time, is a way to modelling and studying complex systems, including the 
intricate neural networks of the brain.

Among the various neuron models available, the Leaky Integrate-and-Fire (LIF) model 
serves as a simplified yet effective way to understand the fundamental properties of 
neurons and their interactions. While the LIF model does not capture the full complexity 
of real neurons, its relative simplicity allows it to be used extensively in large-scale 
simulations of neural networks, making it a widely adopted tool for exploring network dynamics.

One largly influental study of network dynamics in neuroscience [Brunel2000], explored the 
different modes of network activity: asynchronous irregular (AI), asynchronous regular (AR), 
synchronous regular (SR), and synchronous irregular (SI). 

1. **Asynchronous Irregular (AI) Mode:** This represents a state where neural firings are 
scattered randomly in time and across neurons. It is believed to be the mode of activity 
during most wakeful states and has been linked to the processing of complex stimuli and 
behaviors.

2. **Asynchronous Regular (AR) Mode:** In AR mode, neurons fire at a regular interval but 
out of phase with each other, meaning that each neuron fires independently. 
This mode might be involved in independent processing or simultaneous execution of 
separate tasks.

3. **Synchronous Regular (SR) Mode:** In this mode, neurons fire synchronously at 
regular intervals. It is often associated with oscillatory activities in the brain 
such as theta and gamma rhythms, which are important for memory and perception, respectively.

4. **Synchronous Irregular (SI) Mode:** Here, neurons fire synchronously but at irregular 
intervals. This mode of activity is less understood but may be important for certain transient 
brain states or responses to specific stimuli.

Understanding these modes of network activity, and using simulations of LIF neurons to study 
their properties and transitions, is a powerful approach to analyzing complex brain processes. 
It allows for a deeper understanding of network dynamics and how they contribute to various 
cognitive functions, and also provides a useful tool for developing artificial intelligence 
systems that emulate the human brain.

In the following sections, we will delve deeper into the mechanics of LIF neurons, the AI, 
AR, SR, and SI modes of activity, and their implications for network dynamics and 
computational neuroscience.

### Exercise: Simulation of Integrate-and-Fire Neuron Modes

Objective: Simulate and analyze the different modes of operation in an integrate-and-fire 
neuron model.

Implement an integrate-and-fire neuron model in Python. Your neuron should be able to 
summate a large number of synaptic inputs and generate action potentials.

Create two types of inputs for your neuron: excitatory and inhibitory. 
Each input should consist of an independent Poisson spike train driving a synaptic conductance. 
For this exercise, let's consider 1000 excitatory and 200 inhibitory inputs.

Now, simulate two different scenarios to illustrate the two different modes of operation:

a. In the first scenario, the excitatory inputs should be strong enough relative to the 
inhibitory inputs to make the average membrane potential more depolarized than the spiking 
threshold of the model. Record the pattern of action potentials generated by your neuron model 
in this condition.

b. In the second scenario, the average membrane potential should be more hyperpolarized than 
the threshold for action potential generation. Again, record the pattern of action potentials 
generated in this condition.

Plot the membrane potentials for both scenarios. 
You should turn off the action potential generation mechanism of the model for these plots.

Calculate the Coefficient of Variation (CV) for the inter-spike intervals of the output spike 
trains in both scenarios. The CV is the ratio of the standard deviation to the mean of the 
inter-spike intervals.

Analyze and compare the regularity of the spike trains in both scenarios. 
Discuss how the neuron model effectively averages its many synaptic inputs and how this 
impacts the regularity of the output spike train.

Discuss the implications of your findings. How does the mode of operation affect the neuron's 
ability to reflect the temporal properties of fluctuations in total synaptic input? How might 
this relate to the variability seen in in vivo recordings of cortical neurons?

In [1]:
seed(11922)  # to get identical figures for repeated runs

################################################################################
# Model parameters
################################################################################
### General parameters
duration = 1.0*second  # Total simulation time
sim_dt = 0.1*ms        # Integrator/sampling step
N_e = 800             # Number of excitatory neurons
N_i = 200              # Number of inhibitory neurons

### Neuron parameters
u_rest = -60*mV        # Leak reversal potential
g_l = 10*nS          # Leak conductance
E_e = 0*mV             # Excitatory synaptic reversal potential
E_i = -80*mV           # Inhibitory synaptic reversal potential
C_m = 198*pF           # Membrane capacitance
tau_e = 5*ms           # Excitatory synaptic time constant
tau_i = 10*ms          # Inhibitory synaptic time constant
tau_r = 5*ms           # Refractory period
u_th = -50*mV          # Firing threshold

### Synapse parameters
w_e = 0.07*nS          # Excitatory synaptic conductance
w_i = 0.07*nS           # Inhibitory synaptic conductance

################################################################################
# Model definition
################################################################################
# Set the integration time (in this case not strictly necessary, since we are
# using the default value)
defaultclock.dt = sim_dt

### Neurons
neuron_eqs = '''
du/dt = (g_l*(u_rest-u) + g_e*(E_e-u) + g_i*(E_i-u))/C_m    : volt (unless refractory)
dg_e/dt = -g_e/tau_e  : siemens  # post-synaptic exc. conductance
dg_i/dt = -g_i/tau_i  : siemens  # post-synaptic inh. conductance
'''
neurons = NeuronGroup(1, model=neuron_eqs,
                      threshold='u>u_th', reset='u=u_rest',
                      refractory='tau_r', method='euler')
# Random initial membrane potential values and conductances
neurons.u = 'u_rest'
neurons.g_e = 'w_e'
neurons.g_i = 'w_i'

exc_neurons =  PoissonGroup(N_e, 10*Hz)
inh_neurons =  PoissonGroup(N_i, 10*Hz)

exc_syn = Synapses(exc_neurons, neurons, on_pre='g_e += w_e')
inh_syn = Synapses(inh_neurons, neurons, on_pre='g_i += w_i')

exc_syn.connect()
inh_syn.connect()


# ##############################################################################
# # Monitors
# ##############################################################################
# Note that we could use a single monitor for all neurons instead, but in this
# way plotting is a bit easier in the end
exc_mon = SpikeMonitor(exc_neurons)
inh_mon = SpikeMonitor(inh_neurons)


# Record conductances and membrane potential of neuron ni
state_mon = StateMonitor(neurons, ['u', 'g_e', 'g_i'], record=True)

# ##############################################################################
# # Simulation run
# ##############################################################################
run(duration, report='text')

figure(figsize=(12,8))
subplot(2,2,1)
plot(statemon1.t/ms, statemon1.v[0], label='v')
axhline(0.8, ls=':', color='C1', label='threshold')
xlabel('Time (ms)')
ylabel('v')
legend()
title('A - Regular Firing Mode (without spikes)')

subplot(2,2,3)
plot(spikemon1.t/ms, spikemon1.i, '.k')
xlabel('Time (ms)')
ylabel('Spikes')
title('A - Regular Firing Mode (with spikes)')

subplot(2,2,2)
plot(statemon2.t/ms, statemon2.v[0], label='v')
axhline(0.8, ls=':', color='C1', label='threshold')
xlabel('Time (ms)')
ylabel('v')
legend()
title('B - Irregular Firing Mode (without spikes)')

subplot(2,2,4)
plot(spikemon2.t/ms, spikemon2.i, '.k')
xlabel('Time (ms)')
ylabel('Spikes')
title('B - Irregular Firing Mode (with spikes)')

tight_layout()